In [2]:
import pandas as pd
import re
import os

# =====================================================
# CONFIGURATION
# =====================================================

RAW_PATH = r"C:/Users/ssand/OneDrive/Documents/A - Synthea/HealthCare_Analysis dashboard/01_RAW_DATASET"
CLEAN_PATH = r"C:/Users/ssand/OneDrive/Documents/A - Synthea/HealthCare_Analysis dashboard/02_CLEANED_DATASET"

START_DATE = pd.to_datetime("2025-01-01")
END_DATE = pd.to_datetime("2026-01-31")

os.makedirs(CLEAN_PATH, exist_ok=True)

# =====================================================
# UTILITY FUNCTIONS
# =====================================================

def standardize_headers(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )
    return df


def filter_date_range(df, column):
    if column in df.columns:
        # Convert to datetime and remove timezone if present
        df[column] = pd.to_datetime(df[column], errors='coerce').dt.tz_localize(None)

        # Make sure START_DATE & END_DATE are timezone-naive
        start = pd.to_datetime(START_DATE).tz_localize(None)
        end = pd.to_datetime(END_DATE).tz_localize(None)

        df = df[(df[column] >= start) & (df[column] <= end)]

    return df



def convert_datetime_to_date(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.date
    return df


def extract_only_alphabets(value):
    if pd.isna(value):
        return value
    return re.sub(r'[^A-Za-z]', '', str(value))


def expand_marital_status(value):
    mapping = {
        'M': 'Married',
        'S': 'Single',
        'D': 'Divorced',
        'W': 'Widowed'
    }
    return mapping.get(value, value)


def dataset_summary(df, name):
    print(f"\n{name.upper()} SUMMARY")
    print(f"Rows: {df.shape[0]}")
    print(f"Columns: {df.shape[1]}")
    print(f"Column Names: {list(df.columns)}")


def save_cleaned(df, filename):
    df.to_csv(os.path.join(CLEAN_PATH, filename), index=False)


# =====================================================
# 1. ENCOUNTERS (FACT TABLE)
# =====================================================

encounters = pd.read_csv(os.path.join(RAW_PATH, "Encounters.csv"))
encounters = standardize_headers(encounters)
encounters = filter_date_range(encounters, "start")
encounters = convert_datetime_to_date(encounters, ["start", "stop"])

dataset_summary(encounters, "Encounters")
save_cleaned(encounters, "Cleaned_Encounters.csv")


# =====================================================
# 2. PATIENTS (DIMENSION TABLE – NO DATE FILTER)
# =====================================================

patients = pd.read_csv(os.path.join(RAW_PATH, "Patients.csv"))
patients = standardize_headers(patients)

patients = convert_datetime_to_date(patients, ["birthdate", "deathdate"])

# Clean name columns
for col in ["first", "middle", "last", "maiden_name"]:
    if col in patients.columns:
        patients[col] = patients[col].apply(extract_only_alphabets)

# Expand marital status
if "marital" in patients.columns:
    patients["marital"] = patients["marital"].apply(expand_marital_status)

dataset_summary(patients, "Patients")
save_cleaned(patients, "Cleaned_Patients.csv")


# =====================================================
# 3. PROCEDURES (FACT TABLE)
# =====================================================

procedures = pd.read_csv(os.path.join(RAW_PATH, "Procedures.csv"))
procedures = standardize_headers(procedures)
procedures = filter_date_range(procedures, "start")
procedures = convert_datetime_to_date(procedures, ["start", "stop"])

dataset_summary(procedures, "Procedures")
save_cleaned(procedures, "Cleaned_Procedures.csv")


# =====================================================
# 4. PROVIDERS (DIMENSION TABLE)
# =====================================================

providers = pd.read_csv(os.path.join(RAW_PATH, "Providers.csv"))
providers = standardize_headers(providers)

if "name" in providers.columns:
    providers["name"] = providers["name"].apply(extract_only_alphabets)

dataset_summary(providers, "Providers")
save_cleaned(providers, "Cleaned_Providers.csv")


# =====================================================
# 5. OBSERVATIONS (FACT TABLE)
# =====================================================

observations = pd.read_csv(os.path.join(RAW_PATH, "Observations.csv"))
observations = standardize_headers(observations)
observations = filter_date_range(observations, "date")
observations = convert_datetime_to_date(observations, ["date"])

if "code" in observations.columns:
    observations["code"] = observations["code"].str.replace("-", "", regex=False)

dataset_summary(observations, "Observations")
save_cleaned(observations, "Cleaned_Observations.csv")


# =====================================================
# 6. ORGANIZATIONS (DIMENSION TABLE)
# =====================================================

organizations = pd.read_csv(os.path.join(RAW_PATH, "Organizations.csv"))
organizations = standardize_headers(organizations)

if "phone" in organizations.columns:
    organizations["phone"] = (
        organizations["phone"]
        .astype(str)
        .str.split(",").str[0]
        .str.replace("-", "", regex=False)
    )

dataset_summary(organizations, "Organizations")
save_cleaned(organizations, "Cleaned_Organizations.csv")


# =====================================================
# 7. MEDICATIONS (FACT TABLE)
# =====================================================

medications = pd.read_csv(os.path.join(RAW_PATH, "Medications.csv"))
medications = standardize_headers(medications)
medications = filter_date_range(medications, "start")
medications = convert_datetime_to_date(medications, ["start", "stop"])

dataset_summary(medications, "Medications")
save_cleaned(medications, "Cleaned_Medications.csv")


# =====================================================
# 8. CONDITIONS (FACT TABLE)
# =====================================================

conditions = pd.read_csv(os.path.join(RAW_PATH, "Conditions.csv"))
conditions = standardize_headers(conditions)
conditions = filter_date_range(conditions, "start")
conditions = convert_datetime_to_date(conditions, ["start", "stop"])

dataset_summary(conditions, "Conditions")
save_cleaned(conditions, "Cleaned_Conditions.csv")


# =====================================================
# 9. CAREPLANS (FACT TABLE)
# =====================================================

careplans = pd.read_csv(os.path.join(RAW_PATH, "Careplans.csv"))
careplans = standardize_headers(careplans)
careplans = filter_date_range(careplans, "start")
careplans = convert_datetime_to_date(careplans, ["start", "stop"])

dataset_summary(careplans, "Careplans")
save_cleaned(careplans, "Cleaned_Careplans.csv")


print("\n✅ All datasets cleaned and saved successfully!")


ENCOUNTERS SUMMARY
Rows: 934
Columns: 15
Column Names: ['id', 'start', 'stop', 'patient', 'organization', 'provider', 'payer', 'encounterclass', 'code', 'description', 'base_encounter_cost', 'total_claim_cost', 'payer_coverage', 'reasoncode', 'reasondescription']

PATIENTS SUMMARY
Rows: 231
Columns: 28
Column Names: ['id', 'birthdate', 'deathdate', 'ssn', 'drivers', 'passport', 'prefix', 'first', 'middle', 'last', 'suffix', 'maiden', 'marital', 'race', 'ethnicity', 'gender', 'birthplace', 'address', 'city', 'state', 'county', 'fips', 'zip', 'lat', 'lon', 'healthcare_expenses', 'healthcare_coverage', 'income']

PROCEDURES SUMMARY
Rows: 3207
Columns: 10
Column Names: ['start', 'stop', 'patient', 'encounter', 'system', 'code', 'description', 'base_cost', 'reasoncode', 'reasondescription']

PROVIDERS SUMMARY
Rows: 454
Columns: 13
Column Names: ['id', 'organization', 'name', 'gender', 'speciality', 'address', 'city', 'state', 'zip', 'lat', 'lon', 'encounters', 'procedures']

OBSERVATIONS S